In [4]:
import ollama
from langchain_ollama import OllamaLLM
from langchain_community.vectorstores import PGVector
from langchain_ollama import OllamaEmbeddings
from langchain_core.documents import Document
import pandas as pd

# --- 1. กำหนดค่า Ollama ---
model = OllamaLLM(model="llama3.2", base_url="http://localhost:11434")
embeddings = OllamaEmbeddings(model="mxbai-embed-large", base_url="http://localhost:11434")

# --- 2. กำหนดค่าการเชื่อมต่อ PostgreSQL ---
DB_USER = "myuser"
DB_PASSWORD = "mypassword"
DB_HOST = "localhost"
DB_PORT = "5432"
DB_NAME = "mydatabase"
CONNECTION_STRING = f"postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
COLLECTION_NAME = "my_ollama_documents_collection"

# --- 3. ดึงข้อมูลจากไฟล์ CSV และเตรียม LangChain Documents ---
csv_file_path = "my_doc.csv" # ระบุพาธของไฟล์ CSV ของคุณ

try:
    df = pd.read_csv(csv_file_path, encoding='utf-8')
except FileNotFoundError:
    print(f"Error: File not found at {csv_file_path}")
    exit()

docs = []
# กำหนดชื่อคอลัมน์ที่เป็นเนื้อหาหลัก
content_column = 'คำตอบ' # ตรวจสอบชื่อคอลัมน์นี้ให้ตรงกับไฟล์ CSV ของคุณเป๊ะๆ

for index, row in df.iterrows():
    # ตรวจสอบว่าคอลัมน์เนื้อหาหลักไม่ว่าง
    if pd.isna(row[content_column]):
        continue # ข้ามแถวที่ไม่มีเนื้อหาหลัก

    page_content = str(row[content_column])

    # สร้าง metadata จากคอลัมน์อื่นๆ ทั้งหมด ยกเว้นคอลัมน์เนื้อหาหลัก
    metadata = row.drop(content_column).to_dict()

    docs.append(Document(page_content=page_content, metadata=metadata))

# print(f"Loaded {len(docs)} documents from CSV.") # คุณสามารถเปิดบรรทัดนี้เพื่อดูจำนวนเอกสารที่โหลดได้

# --- 4. สร้าง Vector Store และบันทึกข้อมูลลง PostgreSQL ---
print("Connecting to PGVector store and adding documents...")
# 'pre_delete_collection=True' จะลบ collection เก่าก่อนเพิ่มใหม่เสมอ
vectorstore = PGVector.from_documents(
    documents=docs,
    embedding=embeddings,
    collection_name=COLLECTION_NAME,
    connection_string=CONNECTION_STRING,
    pre_delete_collection=True # ตั้งค่าเป็น True ถ้าต้องการลบข้อมูลเก่าใน collection ก่อนเพิ่มใหม่
)
print(f"Documents added to PostgreSQL in collection '{COLLECTION_NAME}'.")

# --- 5. ใช้ Vector Store เป็น Retriever และค้นหา ---
retriever = vectorstore.as_retriever()

# ทำการค้นหาข้อความที่คล้ายคลึงที่สุด
query_text = "อยากเปลี่ยน อีเมล"
retrieved_documents = retriever.invoke(query_text)

print(f"\n--- Retrieved Document for query: '{query_text}' ---")
# แสดงเนื้อหาของเอกสารที่ถูกดึงมา (เฉพาะอันแรก)
if retrieved_documents:
    # แสดงเฉพาะ content ของเอกสารตัวแรกที่ถูกดึงมา
    print(f"Content: {retrieved_documents[0].page_content}")
    # ถ้าคุณอยากได้ metadata ของเอกสารนั้นด้วย ก็สามารถเพิ่มได้แบบนี้:
    # print(f"Metadata: {retrieved_documents[0].metadata}")
else:
    print("No documents found for the query.")

print("\nProcess completed.")

Connecting to PGVector store and adding documents...


c:\Users\stamp7ven\AppData\Local\Programs\Python\Python312\Lib\site-packages\langchain_community\vectorstores\pgvector.py:490: LangChainPendingDeprecationWarning: Please use JSONB instead of JSON for metadata. This change will allow for more efficient querying that involves filtering based on metadata. Please note that filtering operators have been changed when using JSONB metadata to be prefixed with a $ sign to avoid name collisions with columns. If you're using an existing database, you will need to create a db migration for your metadata column to be JSONB and update your queries to use the new operators. 
  store = cls(


Documents added to PostgreSQL in collection 'my_ollama_documents_collection'.

--- Retrieved Document for query: 'อยากเปลี่ยน อีเมล' ---
Content: คุณครูสามารถแก้ไขได้ด้วยตนเองค่ะ ทำตามขั้นตอนดังนี้
1 ล็อกอินเข้าระบบ แล้วมาที่เมนู บัญชี ดังรูป จากนั้นระบบจะแสดงหน้าข้อมูลส่วนตัวค่ะ
2 ทำการแก้ไขข้อมูลอีเมลให้ตรงกับระบบ ENDB จากนั้นกดบันทึก ปุ่มด้านล่างค่ะ และกดปิดหน้าข้อมูลส่วนตัว
3 เมื่อแก้ไขอีเมลเรียบร้อยแล้ว คุณครูสามารถเข้าระบบ ENDB ด้วยอีเมลเดิมได้เลยค่ะ
สอบถามเพิ่มเติมได้ที่ teacherpd@ipst.ac.th พร้อมแจ้งหมายเลขอ้างอิง 52006 เพื่อใช้อ้างอิงและติดตามงานค่ะ

Process completed.
